In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_area(contours,image4):
    # 获取轮廓点集
    contour = contours[-1]

    # 提取横坐标
    x_coordinates = contour[:, 0, 0]

    # 获取最左和最右坐标
    leftmost_x = np.min(x_coordinates)
    rightmost_x = np.max(x_coordinates)
#     print("leftmost_x,rightmost_x:",leftmost_x,rightmost_x)
    image_crop=image4[:,:rightmost_x]
    # 获取图像的高度和宽度
    height, width = image4.shape[:2]
#     print("height_crop, width_crop:",height, width)
    for i in range(len(contours)):
        # 清空画布
        plt.clf()
        is_connected_to_boundary = False

        for point in contours[i]:
            x, y = point[0]

            # 检查轮廓上的某个点是否与图像边界相连
            if cv2.pointPolygonTest(contours[i], (x, y), True) <= 0:
                is_connected_to_boundary = True
                break
        if cv2.isContourConvex(contours[i]) is not None and not is_connected_to_boundary:
            print(True)
        cv2.drawContours(image4, contours, i, (0, 0, 255), 20, 8)
        # 创建一个新的图形
        plt.figure()

        # 将图像显示在图形中
        plt.imshow(image4)

        # 显示图形
        plt.show()

    
    morph_find(image_crop)
    # 将图像转换为HSV颜色空间
    hsv = cv2.cvtColor(image_crop, cv2.COLOR_BGR2HSV)

    # 定义深蓝色范围
    lower_dark_blue = np.array([110, 50, 50])  # 深蓝色下限
    upper_dark_blue = np.array([130, 255, 255])  # 深蓝色上限

    # 创建深蓝色掩码
    dark_blue_mask = cv2.inRange(hsv, lower_dark_blue, upper_dark_blue)

    # 计算非零像素数量
    blue_area = cv2.countNonZero(dark_blue_mask)
#     print(blue_area)

#     # 创建一个新的图形
#     plt.figure()

#     # 将图像显示在图形中
#     plt.imshow(dark_blue_mask)

#     # 显示图形
#     plt.show()
    return blue_area

In [ ]:
def capture_airfoil(file_path):
    image = cv2.imread(file_path)
    # 获取图像的高度和宽度
    height, width = image.shape[:2]
    print("height, width:",height, width)

    # 计算要保留的像素范围
    top_h=int(0.15*height)
    bottom_h=int(0.85*height)

    top_w=int(0.15*width)
    bottom_w=int(0.85*width)

    # 截取图像的上部分
    image = image[top_h:bottom_h, top_w:bottom_w]
    height, width = image.shape[:2]
    # 创建一个新的图形
    plt.figure()

    # 将图像显示在图形中
    plt.imshow(image)

    # 显示图形
    plt.show()

    # 将图像转换为灰度图像
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 对灰度图像进行阈值处理
    _, threshold = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY)
    
    # 连通组件分析
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(threshold)

    # 获取每个区域掩膜的单独最大和最小纵坐标
    for i in range(1, num_labels):  # 跳过背景区域（标签0）
        min_y = stats[i, 1]
        max_y = stats[i, 1] + stats[i, 3]
        if(min_y>0 and max_y<height):
            region_mask = np.uint8(labels == i) * 255  # 创建区域掩膜
            region_image = cv2.bitwise_and(image, image, mask=region_mask)  # 通过按位与操作提取区域图像


    # 使用Canny边缘检测
    edges = cv2.Canny(region_image, 100, 200)

    # 寻找最外侧的轮廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return calculate_area(contours,image)

In [1]:
image4 = cv2.imread("/kaggle/input/flowfieldpic/Pic/case1-100.png")
image4 = cv2.imread("/kaggle/input/flowfieldpic/Pic/case102-240.png")

capture_airfoil(image4)

NameError: name 'cv2' is not defined

In [ ]:
import os
import glob

# 指定文件夹路径
pic_path = glob.glob("/kaggle/input/flowfieldpic/Pic/*")

df = pd.DataFrame(columns=["图片名", '失速面积','失速开始位置'])

# 遍历文件夹中的所有文件
for file_path in pic_path:
    area=capture_airfoil(file_path)
    # 迭代添加行数据
    data = {'图片名': file_name,
            '失速面积': area,
           '失速开始位置':start_coor}
    df.loc[len(df)] = data
    # 保存到Excel文件
    df.to_excel('分类结果.xlsx', index=False)
    

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import seaborn as sns 
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class MNIST_data(Dataset):
    """MNIST dtaa set"""
    
    def __init__(self, file_path, 
                 transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), 
                     transforms.Normalize(mean=(0.5,), std=(0.5,))])
                ):
        
        df = pd.read_csv(file_path)
        
        if len(df.columns) == 784:
            # test data
            self.X = df.values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        else:
            # training data
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:,0].values)
            
        self.transform = transform
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
from PIL import Image

# 定义训练集的转换
train_transform = transforms.Compose([
    transforms.Resize((580, 844)),
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化
])

# 定义测试集的转换
test_transform = transforms.Compose([
    transforms.Resize((580, 844)),
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化
])

# 自定义数据集类
class MyDataset(Dataset):
    def __init__(self, dataframe, test_size=0.2, random_state=42, transform=None):
        self.samples = dataframe['图片名'].values
        self.labels = dataframe['是否失速'].values

        # 根据标签进行分层采样，保持正样本比例一致
        self.train_samples, self.test_samples, self.train_labels, self.test_labels = train_test_split(
            self.samples, self.labels, test_size=test_size, stratify=self.labels, random_state=random_state
        )
        self.transform = transform

    def __len__(self):
        return len(self.train_samples)

    def __getitem__(self, index):
        sample_path = '/kaggle/input/flowfieldpic/Pic/' + self.train_samples[index]
        sample = Image.open(sample_path)
        label = self.train_labels[index]
        if self.transform is not None:
            sample = self.transform(sample)
        # 在这里进行样本和标签的预处理，如果需要的话

        return sample, label

In [ ]:
# 读取包含文件名和属性值的DataFrame
df = pd.read_excel('/kaggle/input/classification-output/output.xlsx')
df_unique=df.drop_duplicates()
# 划分训练集和测试集

df_unique['是否失速'] = df_unique['失速面积'].apply(lambda x: 1 if x > 5000 else 0)

train_dataset = MyDataset(df_unique,transform=train_transform)
test_dataset = MyDataset(df_unique,transform=test_transform)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch.utils.data import DataLoader

In [ ]:
import torch
torch.cuda.empty_cache()  # 清理未使用的GPU内存
torch.cuda.reset_peak_memory_stats()  # 重置最大分配内存

remaining_memory = torch.cuda.max_memory_allocated() - torch.cuda.memory_allocated()
remaining_cache = torch.cuda.max_memory_reserved() - torch.cuda.memory_reserved()

print(f"Remaining memory: {remaining_memory / (1024**2):.2f} MiB")
print(f"Remaining cache: {remaining_cache / (1024**2):.2f} MiB")

In [ ]:
import subprocess as sp
import os

def get_gpu_memory():
    _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]

    ACCEPTABLE_AVAILABLE_MEMORY = 1024
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values
print(get_gpu_memory())

In [ ]:
# 定义分类器模型
class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.resnet = resnet18(weights=True)
        self.fc = nn.Linear(1000, num_classes)

    def forward(self, x):
        features = self.resnet(x)
        output = self.fc(features)
        return output

# 设置超参数
num_classes = 2
lr = 0.001
batch_size = 64
num_epochs = 100

# 创建数据加载器
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

# 创建模型并定义损失函数和优化器
model = Classifier(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


# 模型训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(1)
for epoch in range(num_epochs):
    model.train()
    print(get_gpu_memory())
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 在测试集上评估模型性能
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pt')
    print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.4f}")

In [ ]:

checkpoint = torch.load('model_epoch_19.pt')
model.load_state_dict(checkpoint)

all_files = os.listdir('/kaggle/input/flowfieldpic/Pic')

filtered_files = [file for file in all_files if file not in df['图片名'].tolist()]
# 模型训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# 设置模型为评估模式
model.eval()

df = pd.DataFrame(columns=["图片名", '是否失速'])


# 输入数据进行推理
for file in filtered_files:
#     sample_path = '/kaggle/input/flowfieldpic/Pic/' + file
    sample_path = '/kaggle/input/flowfieldpic/Pic/' + 'case78-235.png'
    sample = Image.open(sample_path)
    transform = transforms.ToTensor()
    tensor_image = transform(sample)
    sample = tensor_image.to(device)
    # 增加一个维度
    output = model(sample.unsqueeze(0))
    _, predicted = torch.max(output.data, 1)
    print(predicted.cpu().numpy())
    data = {'图片名': file,
            '是否失速': predicted.cpu().numpy()}
    df.loc[len(df)] = data
    # 保存到Excel文件
    df.to_excel('classification_output.xlsx', index=False)

In [ ]:
from torch.utils.data import Dataset
import os
from PIL import Image


class MyDataset(Dataset):
    def __init__(self, root, train=True, transform=None):
        super(MyDataset, self).__init__()
        self.root = root
        self.train = train
        self.transform = transform
        imgs = []

        if train:
            imgs = list(zip(os.listdir(root+'/train'), True*len(os.listdir(root+'/train'))))
                
        else:
            imgs = list(zip(os.listdir(root+'/test'), False*len(os.listdir(root+'/test'))))

        self.imgs = imgs

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(self.root+fn).convert('RGB')    # 按路径读取图片

        if self.transform is not None:
            img = self.transform(img)    # 对图片进行 transform 操作
        return img, label

    def __len__(self):
        return len(self.imgs)